In [1]:
cd ..

E:\PROJECTS\Video Membership App\Video-Membership-Webapp


In [2]:
from app import db
from app.users.models import User
from cassandra.cqlengine.management import sync_table

In [3]:
db.get_session()
sync_table(User)

In [16]:
post_data = {
    #"email" : "test@abc",
    "email" : "test@abc.com",
    "password" : "Abcde"
} 

In [23]:
from pydantic import BaseModel,EmailStr, SecretStr, validator

In [18]:
class UserLoginSchema(BaseModel):
    email : EmailStr
    password : SecretStr
        

In [15]:
UserLoginSchema(**post_data)

ValidationError: 1 validation error for UserLoginSchema
email
  value is not a valid email address (type=value_error.email)

In [22]:
UserLoginSchema(**post_data) #email='test@abc.com', password= 'Abcde'

UserLoginSchema(email='test@abc.com', password=SecretStr('**********'))

In [24]:
class UserSignupSchema(BaseModel):
    email : EmailStr
    password : SecretStr
    password_confirm : SecretStr
    
    @validator('email')
    def email_available(cls, v, values, **kwargs): #values = {email: ,password: ,password_confirm:} , v = @validator('***')
        q = User.objects.filter(email = v)
        if q.count() != 0:
            raise ValueError("Email is not available")
        return v
    
    @validator('password_confirm')
    def password_match(cls, v, values, **kwargs):
        password = values.get('password')
        password_confirm = v
        if password != password_confirm:
            raise ValueError('Password does not match')
        return v

In [25]:
UserSignupSchema(email="test@abc.com", password = "Abcde", password_confirm = "Abc")

ValidationError: 2 validation errors for UserSignupSchema
email
  Email is not available (type=value_error)
password_confirm
  Password does not match (type=value_error)

In [26]:
UserSignupSchema(email="test@abc.com", password = "Abcde", password_confirm = "Abcde")

ValidationError: 1 validation error for UserSignupSchema
email
  Email is not available (type=value_error)

In [27]:
UserSignupSchema(email="test1@abc.com", password = "Abcde", password_confirm = "Abcde")

UserSignupSchema(email='test1@abc.com', password=SecretStr('**********'), password_confirm=SecretStr('**********'))